# connect


In [16]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine, MetaData, URL, Identity
from sqlalchemy import Table, Column, Integer, String, BOOLEAN, DATE, ForeignKey


#Creating Database,Connection
meta = MetaData()
USERNAME = 'root'
PASSWORD = '13771377Mnn@'
SERVER = 'localhost'
PORT = 3306
DATABASE = 'book_store'

url_object = URL.create("mysql+mysqlconnector",
                        username=USERNAME,
                        password=PASSWORD,
                        host=SERVER,
                        database=DATABASE)
engine = create_engine(url_object,echo = True)
conn = engine.connect()

2023-09-28 08:32:11,646 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-09-28 08:32:11,667 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:32:11,756 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-09-28 08:32:11,757 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:32:11,760 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-09-28 08:32:11,761 INFO sqlalchemy.engine.Engine [raw sql] {}


# create model database

In [17]:
#Creating Tables
#book
book = Table(
    'book', meta,
    Column('code', Integer, primary_key=True),
    Column('title_persian', String(2000)),
    Column('title_english', String(2000)),
    Column('price', Integer),
    Column('discount', Integer),
    Column('grade', Integer),
    Column('shabak', String(50)),
    Column('page_count', Integer),
    Column('release_year_sh', Integer),
    Column('release_year_mi', Integer),
    Column('exist', BOOLEAN),
    Column('earliest_access', DATE),
    Column('print_series', Integer),
    Column('cover', String(100)),
    Column('ghate', String(100))
)
meta.create_all(engine)

2023-09-28 08:32:16,122 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-28 08:32:16,135 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`book`
2023-09-28 08:32:16,135 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:32:16,320 INFO sqlalchemy.engine.Engine 
CREATE TABLE book (
	code INTEGER NOT NULL AUTO_INCREMENT, 
	title_persian VARCHAR(2000), 
	title_english VARCHAR(2000), 
	price INTEGER, 
	discount INTEGER, 
	grade INTEGER, 
	shabak VARCHAR(50), 
	page_count INTEGER, 
	release_year_sh INTEGER, 
	release_year_mi INTEGER, 
	exist BOOL, 
	earliest_access DATE, 
	print_series INTEGER, 
	cover VARCHAR(100), 
	ghate VARCHAR(100), 
	PRIMARY KEY (code)
)


2023-09-28 08:32:16,323 INFO sqlalchemy.engine.Engine [no key 0.00303s] {}
2023-09-28 08:32:16,448 INFO sqlalchemy.engine.Engine COMMIT


In [18]:
#group
group = Table(
    'group', meta,
    Column('id', Integer, primary_key=True),
    Column('description', String(5000))
)
meta.create_all(engine)

2023-09-28 08:34:30,640 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-28 08:34:30,647 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`book`
2023-09-28 08:34:30,650 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:34:30,676 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group`
2023-09-28 08:34:30,680 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:34:30,684 INFO sqlalchemy.engine.Engine 
CREATE TABLE `group` (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	description VARCHAR(5000), 
	PRIMARY KEY (id)
)


2023-09-28 08:34:30,703 INFO sqlalchemy.engine.Engine [no key 0.01839s] {}
2023-09-28 08:34:30,775 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
#group_book
group_book = Table(
    'group_book', meta,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('group_id', Integer, ForeignKey('group.id')),
    Column('book_code', Integer, ForeignKey('book.code'))
)
meta.create_all(engine)

2023-09-27 00:31:19,640 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-27 00:31:19,642 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`book`
2023-09-27 00:31:19,645 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-27 00:31:19,650 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group`
2023-09-27 00:31:19,651 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-27 00:31:19,656 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group_book`
2023-09-27 00:31:19,657 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-27 00:31:19,662 INFO sqlalchemy.engine.Engine 
CREATE TABLE group_book (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	group_id INTEGER, 
	book_code INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(group_id) REFERENCES `group` (id), 
	FOREIGN KEY(book_code) REFERENCES book (code)
)


2023-09-27 00:31:19,664 INFO sqlalchemy.engine.Engine [no key 0.00383s] {}
2023-09-27 00:31:19,706 INFO sqlalchemy.engine.Engine COMMIT


In [19]:
#person
person = Table(
    'person', meta,
    Column('counter', Integer, primary_key=True,autoincrement=True),
    Column('id', Integer),
    Column('name', String(100)),
    Column('description', String(5000))
)
meta.create_all(engine)

2023-09-28 08:47:55,324 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-28 08:47:55,327 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`book`
2023-09-28 08:47:55,328 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:47:55,331 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group`
2023-09-28 08:47:55,332 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:47:55,337 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`person`
2023-09-28 08:47:55,370 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:47:55,376 INFO sqlalchemy.engine.Engine 
CREATE TABLE person (
	counter INTEGER NOT NULL AUTO_INCREMENT, 
	id INTEGER, 
	name VARCHAR(100), 
	description VARCHAR(5000), 
	PRIMARY KEY (counter)
)


2023-09-28 08:47:55,377 INFO sqlalchemy.engine.Engine [no key 0.00191s] {}
2023-09-28 08:47:55,432 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
#publisher
publisher = Table(
    'publisher', meta,
    Column('id', Integer, primary_key=True),
    Column('name', String(100))
)
meta.create_all(engine)

2023-09-28 08:51:14,603 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-28 08:51:14,608 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`book`
2023-09-28 08:51:14,611 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:51:14,618 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group`
2023-09-28 08:51:14,622 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:51:14,627 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`person`
2023-09-28 08:51:14,628 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:51:14,632 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`publisher`
2023-09-28 08:51:14,634 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:51:14,642 INFO sqlalchemy.engine.Engine 
CREATE TABLE publisher (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(100), 
	PRIMARY KEY (id)
)


2023-09-28 08:51:14,644 INFO sqlalchemy.engine.Engine [no key 0.00288s] {}
2023-09-28 08:51:14,696 INFO sqlalchemy.engine.Engine COMMIT


In [21]:
#category
category = Table(
    'category', meta,
    Column('id', Integer, primary_key=True),
    Column('name', String(150)),
    Column('description', String(5000))
)
meta.create_all(engine)

2023-09-28 08:51:46,450 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-28 08:51:46,488 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`book`
2023-09-28 08:51:46,489 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:51:46,492 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group`
2023-09-28 08:51:46,493 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:51:46,498 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`person`
2023-09-28 08:51:46,500 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:51:46,509 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`publisher`
2023-09-28 08:51:46,512 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:51:46,518 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`category`
2023-09-28 08:51:46,519 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:51:46,533 INFO sqlalchemy.engine.Engine 
CREATE TABLE category (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(150), 
	description VARCHAR(5000), 


In [ ]:
#group_category
group_category = Table(
    'group_category', meta,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('group_id', Integer, ForeignKey('group.id')),
    Column('category_id', Integer, ForeignKey('category.id'))
)
meta.create_all(engine)

In [24]:
#book_publisher
book_publisher = Table(
    'book_publisher', meta,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('book_code', Integer, ForeignKey('book.code')),
    Column('publisher_id', Integer, ForeignKey('publisher.id'))
)
meta.create_all(engine)

2023-09-28 08:56:54,730 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-28 08:56:54,732 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`book`
2023-09-28 08:56:54,734 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:56:54,739 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group`
2023-09-28 08:56:54,741 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:56:54,749 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`person`
2023-09-28 08:56:54,751 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:56:54,758 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`publisher`
2023-09-28 08:56:54,759 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:56:54,763 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`category`
2023-09-28 08:56:54,764 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-28 08:56:54,766 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group_category`
2023-09-28 08:56:54,767 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09

In [ ]:
#crew
crew = Table(
    'crew', meta,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('book_code', Integer, ForeignKey('book.code')),
    Column('person_counter', Integer, ForeignKey('person.counter')),
    Column('role', String(15))
)
meta.create_all(engine)